In [4]:
import pandas as pd
!pip install openpyxl

  Using cached openpyxl-3.0.10-py2.py3-none-any.whl (242 kB)
  Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)


In [2]:
df_papers_1 = pd.read_csv(
    '../data/data_csv/papers_1.csv', 
    sep='\t', encoding='utf-8', index_col='id').dropna().drop_duplicates('title')
df_papers_1.n_citation = df_papers_1.n_citation.astype(int)
df_papers_1 = df_papers_1.query("keywords != '[]'")

In [3]:
df_papers_1.shape

(49111, 14)

In [239]:
df_papers_1.iloc[:1000].head(2)#abstract[:10]

,venue_id,title,abstract,year,n_citation,page_start,page_end,issue,volume,keywords,fos,doi,pdf,url
id,,,,,,,,,,,,,,
2674,1176.0,Introduction.,Our understanding of object recognition can ad...,1986.0,3431,139,141,2,74,"['Sustainable Development', 'Student Teacher',...","['Information system', 'Management information...",10.3928/0090-4481-19750601-05,//static.aminer.org/pdf/PDF/000/750/125/introd...,['http://dx.doi.org/10.3928/0090-4481-19750601...
4064,1572.0,Preface.,On behalf of the IRPS 2010 Management Committe...,2005.0,1,S1,S3,8 Suppl,25,"['remote sensing', 'airborne electromagnetics'...","['Computer science', 'Artificial intelligence']",10.1016/j.bpobgyn.2014.10.013,//static.aminer.org/pdf/PDF/000/868/217/prefac...,['http://dx.doi.org/10.1016/j.bpobgyn.2014.10....


In [138]:
df_papers_1.abstract[:10]

id
2674    Our understanding of object recognition can ad...
4064    On behalf of the IRPS 2010 Management Committe...
4314    This paper describes an approach to the featur...
4397    We investigate the fixed parameter complexity ...
4401    In two-channel competitive genomic hybridizati...
4407    We consider wireless networks that can be mode...
4421    Hierarchy is one of the most conspicuous featu...
4422    This paper studies the problem of determining ...
5876    The evolution of an information society is acc...
7705    Therapeutics in Geriatric Neuropsychiatry Edit...
Name: abstract, dtype: object

In [ ]:
# Create embeddings

In [212]:
%%time
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk, string
import numpy as np
from tqdm import tqdm
nltk.download('punkt') # if necessary...

stemmer = nltk.stem.porter.PorterStemmer()
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)

def stem_tokens(tokens):
    return [stemmer.stem(item) for item in tokens]

def normalize(text):
    return stem_tokens(nltk.word_tokenize(text.lower().translate(remove_punctuation_map)))

SLICE = 1000
map_keys = {k:i  for i,k in zip(df_papers_1.abstract[:SLICE].index, range(len(df_papers_1.abstract[:SLICE].index)))}
corpus = df_papers_1.abstract[:SLICE]  

vect = TfidfVectorizer(tokenizer=normalize, stop_words='english')
tfidf = vect.fit_transform(corpus)   
                                                                                                                                                                                                                    
pairwise_similarity = tfidf * tfidf.T

#view the pairwise similarities 
# print(pairwise_similarity)

#check how a string is normalized
print(normalize("The scikit-learn docs are Orange and Blue"))

[nltk_data] Error loading punkt: <urlopen error [Errno 99] Cannot
[nltk_data]     assign requested address>


['the', 'scikitlearn', 'doc', 'are', 'orang', 'and', 'blue']
CPU times: user 17.2 s, sys: 6.56 ms, total: 17.2 s
Wall time: 17.3 s


In [ ]:
# Get list of similarities

In [217]:
collector = {}
for item in tqdm(pairwise_similarity):
    map_list = [(k,v) for k,v in zip(item.indices, item.data)]
    similar = [(map_keys[item[0]], item[1]) for item in map_list if 0.1 < item[1] < 0.99]
    if len(item.indices) > 0:
        collector[map_keys[item.indices[-1]]] = similar


1000it [00:02, 401.71it/s]


In [ ]:
# Choose some title

In [235]:
tit = 'Ergodic mirror descent.'
call = df_papers_1[df_papers_1.title==tit].index[0]
book_list = collector[call]
book_list.sort(key=lambda x:x[1], reverse=True)

In [ ]:
# Get recomedation by n_citation and similarity

In [280]:
idx = [itm[0] for itm in book_list]
map_similar = {itm[0]:itm[1] for itm in book_list}
df_local = df_papers_1[df_papers_1.index.isin(idx)]
df_local.loc[:,'similarity'] = [map_similar[it] for it in df_local.index]
df_local = df_local[df_local.index.isin(idx)].sort_values(['n_citation', 'similarity'])[::-1]
df_local.iloc[:3]

/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


,venue_id,title,abstract,year,n_citation,page_start,page_end,issue,volume,keywords,fos,doi,pdf,url,similarity
id,,,,,,,,,,,,,,,
23474,5968.0,Biogeography-Based Optimization,Biogeography is the study of the geographical ...,2008.0,2104,702,713,6,12,"['particle swarm optimization', 'biology-based...","['Particle swarm optimization', 'Mathematical ...",10.1109/TEVC.2008.919004,https://static.aminer.cn/upload/pdf/program/53...,['http://dx.doi.org/10.1109/TEVC.2008.919004'],0.120411
33581,2822.0,Extremely randomized trees,This paper proposes a new tree-based ensemble ...,2006.0,1900,3,42,1,63,"['Supervised learning', 'Decision and regressi...","['Cut-point', 'Kernel (linear algebra)', 'Deci...",10.1007/s10994-006-6226-1,//static.aminer.org/pdf/PDF/001/009/448/extrem...,['http://dx.doi.org/10.1007/s10994-006-6226-1'...,0.100472
25072,450.0,Annealed importance sampling,Simulated annealing—moving from a tractable di...,2001.0,954,125,139,2,11,"['tempered transitions', 'sequential importanc...","['Slice sampling', 'Statistical physics', 'Pro...",10.1023/A:1008923215028,https://static.aminer.cn/storage/pdf/arxiv/98/...,"['http://dx.doi.org/10.1023/A:1008923215028', ...",0.158943


In [1]:
# Additional work

In [13]:
df_authors = pd.read_csv('../data/data_csv/sample_author_paper_22.csv', sep='\t', index_col='id')

In [11]:
df_authors.paper_id.value_counts()

1929590    35
3030917    28
1017870    27
1553088    24
4604627    24
           ..
869240      1
4340738     1
3137743     1
868254      1
2630571     1
Name: paper_id, Length: 9992, dtype: int64

In [15]:
df_paper = pd.read_csv('../data/data_csv/sampled_papers_22.csv', sep='\t', index_col='id')

In [17]:
# len(set(df_paper.index) & set(df_authors.index))

39

In [19]:
pd.read_csv('../data/data_csv/sample_papers_refs_22.csv', sep='\t', index_col='id')

,parent_paper,child_paper
id,,
0,46400,705578
1,66869,840203
2,85818,1659843
3,92078,166670
4,105843,2088805
...,...,...
288,5113913,4907523
289,5131294,3683706
290,5169277,4144432


In [31]:
df_paper.loc[634356].title

'Predicting temporal errors in complex task environments: A computational and experimental approach'